## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-17-10-40-00 +0000,wsj,Fiscal Credibility Concerns Remain Headwind Fo...,https://www.wsj.com/finance/currencies/asian-c...
1,2025-11-17-10-35-33 +0000,bbc,Moment Ex-Bangladesh leader Sheikh Hasina's se...,https://www.bbc.com/news/videos/c8drzn7g393o?a...
2,2025-11-17-10-34-30 +0000,nyt,"Too Powerful to Ignore, Saudi Prince Returns t...",https://www.nytimes.com/2025/11/17/world/middl...
3,2025-11-17-10-30-42 +0000,nyt,China Escalates Feud With Japan on Military an...,https://www.nytimes.com/2025/11/17/world/asia/...
4,2025-11-17-10-30-00 +0000,wsj,L.A. Residents Are Obsessed With Their New B-L...,https://www.wsj.com/us-news/black-bears-los-an...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2334/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
58,trump,33
33,new,14
63,epstein,13
64,files,9
335,will,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
65,2025-11-17-04-47-00 +0000,wsj,Trump threw in the towel on dissuading House R...,https://www.wsj.com/politics/policy/trump-play...,96
79,2025-11-17-02-44-24 +0000,nypost,Trump says House Republicans should vote to re...,https://nypost.com/2025/11/16/us-news/trump-sa...,80
186,2025-11-16-16-58-43 +0000,nypost,MTG suggests Israel or ‘other foreign gov’t’ m...,https://nypost.com/2025/11/16/us-news/mtg-sugg...,76
8,2025-11-17-10-21-07 +0000,nyt,Trump Says House Republicans Should Vote to Re...,https://www.nytimes.com/2025/11/16/us/politics...,75
47,2025-11-17-07-57-07 +0000,bbc,Trump calls on House Republicans to vote to re...,https://www.bbc.com/news/articles/cvgv653v1vjo...,72


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
65,96,2025-11-17-04-47-00 +0000,wsj,Trump threw in the towel on dissuading House R...,https://www.wsj.com/politics/policy/trump-play...
199,39,2025-11-16-13-28-00 +0000,wsj,New Rare-Earths Plant in Europe Shows How Toug...,https://www.wsj.com/world/europe/a-new-rare-ea...
143,38,2025-11-16-21-02-08 +0000,latimes,Body of 7-year-old girl is found off Big Sur; ...,https://www.latimes.com/california/story/2025-...
109,35,2025-11-17-00-19-40 +0000,nypost,"Liberal pols, DSA warn Gov. Kathy Hochul to ba...",https://nypost.com/2025/11/16/us-news/liberals...
122,34,2025-11-16-23-33-06 +0000,nypost,Vile antisemitic graffiti scrawled on Mexico’s...,https://nypost.com/2025/11/16/world-news/antis...
25,29,2025-11-17-09-57-22 +0000,cbc,Bangladesh's ousted PM Sheikh Hasina sentenced...,https://www.cbc.ca/news/world/bangladesh-s-ous...
185,27,2025-11-16-17-00-00 +0000,wsj,Over 60% of Americans are comfortable with how...,https://www.wsj.com/politics/policy/economic-o...
81,25,2025-11-17-02-30-26 +0000,nypost,Trump says Venezuela would ‘like to talk’ — as...,https://nypost.com/2025/11/16/world-news/trump...
41,23,2025-11-17-09-00-00 +0000,cbc,"Budget, government’s fate up in the air as MPs...",https://www.cbc.ca/news/politics/budget-canada...
177,23,2025-11-16-18-07-31 +0000,nypost,9 people killed in powerful explosion at India...,https://nypost.com/2025/11/16/world-news/9-peo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
